In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
df=pd.read_csv("/content/drive/MyDrive/KDD_Train (1).csv")

In [4]:
print(df.shape)
df.head(5)
df = df.reset_index()

(125973, 42)


In [5]:
df.dropna()

,index,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,0,tcp,ftp_data,SF,491,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,1,0,udp,other,SF,146,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,2,0,tcp,private,S0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,3,0,tcp,http,SF,232,8153,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,4,0,tcp,http,SF,199,420,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,125968,0,tcp,private,S0,0,0,0,0,0,...,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
125969,125969,8,udp,private,SF,105,145,0,0,0,...,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125970,125970,0,tcp,smtp,SF,2231,384,0,0,0,...,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125971,125971,0,tcp,klogin,S0,0,0,0,0,0,...,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly


In [6]:
df.value_counts('protocol_type')
df.value_counts('flag')

flag
SF        74945
S0        34851
REJ       11233
RSTR       2421
RSTO       1562
S1          365
SH          271
S2          127
RSTOS0      103
S3           49
OTH          46
dtype: int64

In [7]:
df.replace(('normal','anomaly'),(0,1),inplace=True)
df.replace(('tcp','udp','icmp'),(0,1,2),inplace=True)
df=df[df.columns.drop('service')]
df=df[df.columns.drop('flag')]
df.head()

,index,duration,protocol_type,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,0,0,491,0,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,0
1,1,0,1,146,0,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,0
2,2,0,0,0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,1
3,3,0,0,232,8153,0,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,0
4,4,0,0,199,420,0,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0


In [8]:
y=df['class']
y = np.nan_to_num(y)
X=df[df.columns.drop('class')]

# **K-FOLD CROSS VALIDATION FUNCTION**

In [48]:
from sklearn.model_selection import cross_validate
def cross_validation(model, _X, _y, _cv=5):
      '''Function to perform 5 Folds Cross-Validation
       Parameters
       ----------
      model: Python Class, default=None
              This is the machine learning algorithm to be used for training.
      _X: array
           This is the matrix of features.
      _y: array
           This is the target variable.
      _cv: int, default=5
          Determines the number of folds for cross-validation.
       Returns
       -------
       The function returns a dictionary containing the metrics 'accuracy', 'precision',
       'recall', 'f1' for both training set and validation set.
      '''
      _scoring = ['accuracy', 'precision', 'recall', 'f1']
      results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)
      
      return {"Mean Training Accuracy": results['train_accuracy'].mean()*100,
              "Mean Training Precision": results['train_precision'].mean()*100,
              "Mean Training Recall": results['train_recall'].mean()*100}

# **APPLYING MRMR FOR FEATURE EXTRACTION**

In [9]:
!pip install mrmr_selection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 763 kB/s 
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=8616fa5df9779b204ad013f06b3e03a4881cbe8681c3aa42054ad3d9d493ef12
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built sklearn


In [10]:
from mrmr import mrmr_classif
selected_features = mrmr_classif(X=X, y=y, K=10)

100%|██████████| 10/10 [00:02<00:00,  4.42it/s]


In [11]:
print(selected_features)

['same_srv_rate', 'dst_host_srv_count', 'logged_in', 'dst_host_srv_serror_rate', 'dst_host_same_srv_rate', 'serror_rate', 'count', 'dst_host_serror_rate', 'srv_serror_rate', 'dst_host_count']


In [12]:
X1=X[selected_features]
X1 = np.nan_to_num(X1)
len(X1)

125973

**5-FOLD CROSS VALIDATION**

DECISION TREE

In [46]:
from sklearn.tree import DecisionTreeClassifier
decision_tree_model = DecisionTreeClassifier(criterion="entropy",random_state=0)
decision_tree_result = cross_validation(decision_tree_model,X1,y,5)
print(decision_tree_result)

{'Mean Training Accuracy': 97.88665025955575, 'Mean Training Precision': 97.79340417479452, 'Mean Training Recall': 97.66288589459322}


K-NEAREST NEIGHBOURS

In [51]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn_result = cross_validation(knn,X1,y,5)
print(knn_result)

{'Mean Training Accuracy': 95.91995897061022, 'Mean Training Precision': 95.20009870530448, 'Mean Training Recall': 96.08135766672353}


NAIVE BAYES

In [52]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
nb_result = cross_validation(gnb,X1,y,5)
print(nb_result)

{'Mean Training Accuracy': 88.37945437843976, 'Mean Training Precision': 93.8765184014207, 'Mean Training Recall': 80.26778099948831}


**10-FOLD CROSS VALIDATION**

DECISION TREE

In [53]:
from sklearn.tree import DecisionTreeClassifier
decision_tree_model = DecisionTreeClassifier(criterion="entropy",random_state=0)
decision_tree_result = cross_validation(decision_tree_model,X1,y,10)
print(decision_tree_result)

{'Mean Training Accuracy': 97.8418655953429, 'Mean Training Precision': 97.72459293718677, 'Mean Training Recall': 97.63640153884057}


K-NEAREST NEIGHBOURS

In [54]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn_result = cross_validation(knn,X1,y,10)
print(knn_result)

{'Mean Training Accuracy': 95.96694881999414, 'Mean Training Precision': 95.16636841475587, 'Mean Training Recall': 96.22358671139159}


NAIVE BAYES

In [55]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
nb_result = cross_validation(gnb,X1,y,10)
print(nb_result)

{'Mean Training Accuracy': 88.37740367281664, 'Mean Training Precision': 93.87148801286529, 'Mean Training Recall': 80.26797051187295}


**15-FOLD CROSS VALIDATION**

DECISION TREE

In [56]:
from sklearn.tree import DecisionTreeClassifier
decision_tree_model = DecisionTreeClassifier(criterion="entropy",random_state=0)
decision_tree_result = cross_validation(decision_tree_model,X1,y,15)
print(decision_tree_result)

{'Mean Training Accuracy': 97.8283895286144, 'Mean Training Precision': 97.69755364941946, 'Mean Training Recall': 97.6350479262702}


K-NEAREST NEIGHBOURS

In [57]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn_result = cross_validation(knn,X1,y,15)
print(knn_result)

{'Mean Training Accuracy': 95.97300329787313, 'Mean Training Precision': 95.14212770466028, 'Mean Training Recall': 96.26397944802746}


NAIVE BAYES

In [58]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
nb_result = cross_validation(gnb,X1,y,15)
print(nb_result)

{'Mean Training Accuracy': 88.37846208163992, 'Mean Training Precision': 93.87204808199594, 'Mean Training Recall': 80.26985213550407}


# **APPLYING LASSO FOR FEATURE EXTRACTION**

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso

In [17]:
X= np.nan_to_num(X)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [19]:
pipeline = Pipeline([('scaler',StandardScaler()),('model',Lasso())])

In [20]:
search = GridSearchCV(pipeline,{'model__alpha':np.arange(0.1,10,0.1)},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3)

In [21]:
search.fit(X_train,y_train)

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 1/5] END .................model__alpha=0.1;, score=-0.090 total time=   0.5s
[CV 2/5] END .................model__alpha=0.1;, score=-0.089 total time=   0.3s
[CV 3/5] END .................model__alpha=0.1;, score=-0.089 total time=   0.4s
[CV 4/5] END .................model__alpha=0.1;, score=-0.089 total time=   0.3s
[CV 5/5] END .................model__alpha=0.1;, score=-0.090 total time=   0.2s
[CV 1/5] END .................model__alpha=0.2;, score=-0.131 total time=   0.2s
[CV 2/5] END .................model__alpha=0.2;, score=-0.130 total time=   0.1s
[CV 3/5] END .................model__alpha=0.2;, score=-0.130 total time=   0.1s
[CV 4/5] END .................model__alpha=0.2;, score=-0.131 total time=   0.1s
[CV 5/5] END .................model__alpha=0.2;, score=-0.131 total time=   0.1s
[CV 1/5] END .model__alpha=0.30000000000000004;, score=-0.195 total time=   0.2s
[CV 2/5] END .model__alpha=0.30000000000000004;

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1, 5.2,
       5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4, 6.5,
       6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8,
       7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. , 9.1,
       9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='neg_mean_squared_error', verbose=3)

In [22]:
search.best_params_

{'model__alpha': 0.1}

In [23]:
coefficients = search.best_estimator_.named_steps['model'].coef_

In [24]:
importance = np.abs(coefficients)

In [25]:
features=[]
for col in df.columns:
    if(col!='class'):
      features.append(col)

In [26]:
selected_lasso=np.array(features)[importance > 0]
new_lasso=selected_lasso.tolist()
print(new_lasso)

['logged_in', 'count', 'same_srv_rate', 'dst_host_srv_count', 'dst_host_srv_serror_rate']


In [40]:
X2 = df[new_lasso]
X2 = np.nan_to_num(X2)
len(X2)

125973

**5-FOLD CROSS VALIDATION**

DECISION TREE

In [59]:
from sklearn.tree import DecisionTreeClassifier
decision_tree_model = DecisionTreeClassifier(criterion="entropy",random_state=0)
decision_tree_result = cross_validation(decision_tree_model,X2,y,5)
print(decision_tree_result)

{'Mean Training Accuracy': 94.64905173709406, 'Mean Training Precision': 95.0889790462424, 'Mean Training Recall': 93.3229575302746}


K-NEAREST NEIGHBOURS

In [60]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn_result = cross_validation(knn,X2,y,5)
print(knn_result)

{'Mean Training Accuracy': 93.20747335115115, 'Mean Training Precision': 93.50556674366042, 'Mean Training Recall': 91.79899368923759}


NAIVE BAYES

In [61]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
nb_result = cross_validation(gnb,X2,y,5)
print(nb_result)

{'Mean Training Accuracy': 88.30185832184307, 'Mean Training Precision': 93.22953631678953, 'Mean Training Recall': 80.72786969128434}


**10-FOLD CROSS VALIDATION**

DECISION TREE

In [62]:
from sklearn.tree import DecisionTreeClassifier
decision_tree_model = DecisionTreeClassifier(criterion="entropy",random_state=0)
decision_tree_result = cross_validation(decision_tree_model,X2,y,10)
print(decision_tree_result)

{'Mean Training Accuracy': 94.58949315792961, 'Mean Training Precision': 95.02751348902443, 'Mean Training Recall': 93.25468569371009}


K-NEAREST NEIGHBOURS

In [63]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn_result = cross_validation(knn,X2,y,10)
print(knn_result)

{'Mean Training Accuracy': 93.34442863951186, 'Mean Training Precision': 93.87181489661913, 'Mean Training Recall': 91.69253510716926}


NAIVE BAYES

In [64]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
nb_result = cross_validation(gnb,X2,y,10)
print(nb_result)

{'Mean Training Accuracy': 88.30304909660836, 'Mean Training Precision': 93.23165371368268, 'Mean Training Recall': 80.72848560653438}


**15-FOLD CROSS VALIDATION**

DECISION TREE

In [65]:
from sklearn.tree import DecisionTreeClassifier
decision_tree_model = DecisionTreeClassifier(criterion="entropy",random_state=0)
decision_tree_result = cross_validation(decision_tree_model,X2,y,15)
print(decision_tree_result)

{'Mean Training Accuracy': 94.57083207118, 'Mean Training Precision': 95.00503871882788, 'Mean Training Recall': 93.23688518067937}


K-NEAREST NEIGHBOURS

In [66]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn_result = cross_validation(knn,X2,y,15)
print(knn_result)

{'Mean Training Accuracy': 93.36564187805094, 'Mean Training Precision': 93.5022701027952, 'Mean Training Recall': 92.15686945163941}


NAIVE BAYES

In [67]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
nb_result = cross_validation(gnb,X2,y,15)
print(nb_result)

{'Mean Training Accuracy': 88.30350270169983, 'Mean Training Precision': 93.23239331534025, 'Mean Training Recall': 80.7287834295751}
